<a href="https://colab.research.google.com/github/mdeniz20/NLP-0/blob/main/ai_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Chain

In [ ]:
!pip install -U langchain-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnableSequence
import getpass
import os
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [ ]:
class TestLangchain:
    def add_one(self, x):
        return x + 1
    def multiply_by_two(self, x):
        return x * 2


    def run(self, x):
      runnable1 = RunnableLambda(self.add_one)
      runnable2 = RunnableLambda(self.multiply_by_two)
      sequence = runnable1 | runnable2
      return sequence.invoke(x)

    def run_alt(self, x):
      runnable1 = RunnableLambda(lambda x: x + 1)
      runnable2 = RunnableLambda(lambda x: x * 2)
      sequence = runnable1 | runnable2
      return sequence.invoke(x)

    def run_alt2(self, x):
      runnable1 = RunnableLambda(lambda x: x + 1)
      runnable2 = RunnableLambda(lambda x: self.multiply_by_two(x))
      runnable3 = RunnableLambda(lambda x: f"The result is {x}")
      sequence = RunnableSequence(first =  runnable1, middle = [runnable2], last=runnable3)
      return sequence.invoke(x)

test_chain = TestLangchain()
print(test_chain.run(2))
print(test_chain.run_alt(2))
print(test_chain.run_alt2(2))


6
6
The result is 6


In [ ]:
class Conversation:
  __messages = []
  __chat = None
  def __init__(self, *, api_key: str = "", model: str = "command-light", system_message: str = ""):
    if system_message:
      self.__add_message(SystemMessage(content=system_message))
    if api_key:
      os.environ["COHERE_API_KEY"] = api_key
    else:
      os.environ["COHERE_API_KEY"] = getpass.getpass()

    self.__chat = ChatCohere(model=model)

  def set_model(self, model: str) -> None:
    self.__chat = ChatCohere(model=model)
  def get_model(self) -> str:
    return self.__chat.model

  def __add_message(self, message) -> None:
    self.__messages.append(message)

  def add_message(self, message) -> None:
    self.__add_message(message)
  def get_messages(self):
    return self.__messages

  def invoke_chat(self, message) -> str:
    self.__add_message(HumanMessage(content=message))
    response = self.__chat.invoke(self.__messages)
    self.__add_message(AIMessage(content=response.content))
    return response

  def chat(self, message) -> str:
    return self.invoke_chat(message).content
ai = Conversation()

··········


In [ ]:
ai.set_model("command-r-plus")
print(ai.chat("how do you find those jokes?"))

As an AI language model, I have been trained on vast amounts of text data, including jokes, stories, books, and conversations. When you ask me to tell a joke, I search my database for relevant humor and select the most appropriate response based on the context and your request. 

It's an automated process where I analyze the input, process the request, and generate a response that aligns with your query. In this case, I searched for jokes related to lawyers and computer engineers and presented the ones that matched the criteria. 

So, when you ask me to "tell a joke," I'm happy to oblige and provide some laughter and entertainment!


In [ ]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.chat_models import ChatCohere

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.pydantic_v1 import BaseModel, Field

os.environ["TAVILY_API_KEY"] = "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"

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."

class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = TavilySearchInput

# LLM
llm = ChatCohere(model="command-r-plus", temperature=0)

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource. You are equipped with an internet search tool and a special vectorstore of information about how to write good essays.
"""

# Prompt template
prompt = ChatPromptTemplate.from_template("How much tl needed to buy one dollar?")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=llm,
    tools=[internet_search],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

response = agent_executor.invoke({
    "input": "I want to write an essay. Any tips?",
    "preamble": preamble,
})

print(response['output'])



> Entering new AgentExecutor chain...

I will search for the current exchange rate between the dollar and the Turkish lira.
{'tool_name': 'internet_search', 'parameters': {'query': 'exchange rate dollar to turkish lira'}}
[{'url': 'https://www.xe.com/currencyconverter/convert/?From=USD&To=TRY', 'content': '1 USD to TRY - Convert US Dollars to Turkish Lire\nXe Currency Converter\n1.00 US Dollar =\n28.904216 Turkish Lire\n1 TRY = 0.0345970 USD\nConvert US Dollar to Turkish Lira\nConvert Turkish Lira to US Dollar\nUSD to TRY Chart\n1 USD = 0 TRY\n1 US Dollar to Turkish Lira stats\n Popular US Dollar (USD) Currency Pairings\nUSD to USD\nUSD to EUR\nUSD to GBP\nUSD to JPY\nUSD to CAD\nUSD to AUD\nUSD to CHF\nUSD to CNY\n Over 70 million downloads worldwide\n4.5/5, 2.2k ratings\n3.8/5, 90.8k ratings\n4.7/5, 41.5k ratings\nLanguage\nTransfer Money\nApps\nTools & Resources\nXE Business\nCompany Info\nConsent Manager The currency symbol is $.\nTRY - Turkish Lira\nOur currency rankings show th

# Logger Class

In [23]:
import os
from datetime import datetime, timedelta, timezone

class Logger:
    """Helps you to trace what is happening in your code"""

    def __init__(self, dir_name: str):
        # Ensure the directory exists
        os.makedirs(dir_name, exist_ok=True)
        self.dir_name = dir_name

        current_time = self.get_current_time()
        filename = f"{current_time}.txt"
        self.file_path = os.path.join(dir_name, filename)

        # Create the file if it doesn't exist
        with open(self.file_path, 'w') as _:
            pass

    def log(self, input: str) -> None:
        with open(self.file_path, 'a') as file:
            current_time = self.get_current_time("%H:%M:%S")
            file.write(f"{current_time} -> {input}\n")

    def get_current_time(self, format="%d-%m-%Y_%H:%M:%S") -> str:
        # Get current date and time with timezone information
        current_date_time = datetime.now(timezone.utc)

        # Turkey is UTC+3
        turkey_time = current_date_time + timedelta(hours=3)

        return turkey_time.strftime(format)

    def clear_all_in_dir(self) -> None:
        for filename in os.listdir(self.dir_name):
            file_path = os.path.join(self.dir_name, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)


# Clear Logs

In [24]:
Logger("logs").clear_all_in_dir()

# Installations

In [25]:
!pip install langchainhub
!pip install -U langchain-cohere
!pip install wikipedia
%pip install --upgrade --quiet  langchain-community
!pip install mysql-connector-python
!apt-get -y install mysql-server
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root';FLUSH PRIVILEGES;"
!pip install langchain langchain-community chromadb cohere
!pip install langchain_cohere
!pip install -U langchain-chroma
!pip install -U sentence-transformers
!pip install huggingface
!pip install langchain_huggingface
!pip install gradio
!pip install chromadb

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 5

# Imports

In [26]:
import os
import mysql.connector
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import Tool, StructuredTool
from langchain_cohere import ChatCohere
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.schema import SystemMessage
from pydantic import BaseModel
import requests
import datetime
from datetime import timezone, timedelta
import base64
import uuid
from enum import Enum
import gradio as gr
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_cohere import CohereEmbeddings
import requests
from google.colab import userdata
from langchain_huggingface import HuggingFaceEmbeddings
import imaplib
import email
from datetime import datetime
import pytz
from binascii import Error  # Not needed unless you're working with binary data; consider removing if not used

In [9]:
from IPython.display import display, HTML
import random
from urllib.parse import quote

# Environment Variables

In [27]:
from google.colab import userdata

os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
os.environ["WEATHER_API_KEY"] = userdata.get('weather_api')
os.environ["ALPHAVANTAGE_STOCK_API_KEY"] = userdata.get('ALPHAVANTAGE_STOCK_API_KEY')
os.environ["AV_EXCH_API_KEY"] = userdata.get('AV_EXCH_API_KEY')
os.environ["STABLE_API_KEY"] = userdata.get('STABLE_API_KEY')
os.environ["GMAIL_APP_PASSWORD"] = userdata.get('GMAIL_APP_PASSWORD')


# Email Handler

In [28]:
class Environment:
  def set_env(self):
    path = os.path.join(os.getcwd(), "security_keys")
    if not os.path.exists(path):
      os.mkdir(path)
    email_security_keys_path = os.path.join(path, "email.txt")
    if not os.path.exists(email_security_keys_path):
      with open(email_security_keys_path, "w") as f:
        f.write("mibrdeniz@gmail.com\n"+userdata.get("GMAIL_APP_PASSWORD"))
env = Environment()
env.set_env()

In [29]:
class EmailHandler:

  class EmailHost:
      host = {
          "GMAIL": "imap.gmail.com",
          "OUTLOOK": "outlook.office365.com",
          "HOTMAIL": "outlook.live.com"
      }

  def __init__(self, email: str = None, password: str = None, file_path: str = None, host: str = "GMAIL"):
      self.email = email
      self.password = password
      self.file_path = file_path

      if not email and not password:
          self.email, self.password, self.file_path = self._get_email_details_from_file(file_path)

      if not self.email or not self.password:
          raise ValueError("Email or password not set")

      host = host.upper()
      if host not in self.EmailHost.host:
          raise ValueError(f"Host {host} not supported")
      self.mail_host = self.EmailHost.host[host]

      self.mail = None

  def _login(self):
      self.mail = imaplib.IMAP4_SSL(self.mail_host)
      try:
          # Login
          self.mail.login(self.email, self.password)
      except imaplib.IMAP4.error as e:
          raise Error(f"Login failed: {e}")

      status, _ = self.mail.select("INBOX")
      if status != 'OK':
          self.mail.logout()
          raise Error(f"Failed to select inbox: {status}")

  def _get_email_details_from_file(self, file_path: str):
      if not file_path:
          path = os.path.join(os.getcwd(), "security_keys")
          email_security_keys_path = os.path.join(path, "email.txt")
      else:
          email_security_keys_path = file_path

      if not os.path.exists(email_security_keys_path):
          raise FileNotFoundError(f"Security keys file not found {email_security_keys_path}")

      with open(email_security_keys_path, "r") as f:
          lines = f.readlines()
          email = lines[0].strip()
          password = lines[1].strip()
      return email, password, email_security_keys_path

  def get_todays_emails(self)->list:
      self._login()
      mail_list = []
      istanbul_tz = pytz.timezone('Europe/Istanbul')

      # Get today's date in Istanbul timezone and convert to UTC
      local_now = datetime.now(istanbul_tz)
      utc_now = local_now.astimezone(pytz.UTC)
      today_date_utc = utc_now.strftime("%d-%b-%Y")

      try:
          # Search for emails from today in UTC
          status, selected_mails = self.mail.search(None, f'(ON "{today_date_utc}")')
          if status != 'OK':
              raise Error(f"Failed to search emails: {status}")

          # Total number of mails from today
          num_of_mails = len(selected_mails[0].split())
          counter = 1

          for num in selected_mails[0].split():
              try:
                  status, data = self.mail.fetch(num, '(RFC822)')
                  if status != 'OK':
                      raise Error(f"Failed to fetch email {num}: {status}")

                  _, bytes_data = data[0]
                  email_message = email.message_from_bytes(bytes_data)

                  subject = email_message.get('subject', '')
                  to = email_message.get('to', '')
                  from_ = email_message.get('from', '')
                  date = email_message.get('date', '')

                  body = ""
                  for part in email_message.walk():
                    if part.get_content_type() in ("text/plain", "text/html"):
                        message = part.get_payload(decode=True)
                        try:
                            # Try decoding with UTF-8 first
                            body = message.decode('utf-8')
                        except UnicodeDecodeError:
                            try:
                                # If UTF-8 decoding fails, try Latin-1
                                body = message.decode('latin-1')
                            except UnicodeDecodeError:
                                # Handle case where neither encoding works
                                body = message.decode('utf-8', errors='replace')  # You could also use 'ignore'
                        break


                  _mail = {
                      "subject": subject,
                      "to": to,
                      "from": from_,
                      "date": date,
                      "body": body
                  }
                  mail_list.append(_mail)

              except imaplib.IMAP4.error as e:
                  print(f"Error during fetch: {e}")
                  continue

      except imaplib.IMAP4.error as e:
          print(f"Error during search: {e}")

      finally:
          if self.mail:
              self.mail.logout()

      return mail_list
  from urllib.parse import quote

  def get_mailto_link(self, to, subject='', body='', cc='', bcc=''):
      """
      Opens the default email client with pre-filled fields.

      Parameters:
          to (str): Recipient email address.
          subject (str): Subject of the email.
          body (str): Body of the email.
          cc (str): CC email addresses (comma-separated).
          bcc (str): BCC email addresses (comma-separated).
      """

      base = "mailto:"
      params = []

      if to:
          params.append(f"{to}")

      if subject:
          params.append(f"subject={quote(subject)}")

      if body:
          params.append(f"body={quote(body)}")

      if cc:
          params.append(f"cc={quote(cc)}")

      if bcc:
          params.append(f"bcc={quote(bcc)}")

      # Combine base with the query parameters
      if params:
          mailto_link = base + '?' + '&'.join(params[1:])  # Use '?' to start parameters and skip the first element
          mailto_link = base + params[0] + ('?' + '&'.join(params[1:]) if len(params) > 1 else '')
      else:
          mailto_link = base

      # Log the link (assuming logger is properly configured elsewhere in your code)
      logger.log(mailto_link)

      return mailto_link



  def __del__(self):
      # Ensure logout is called when the object is being destroyed
      if self.mail:
          self.mail.logout()

In [30]:
email_handl = EmailHandler()
email_address = "mr.mehmet@example.com"
subject = "test"
body = "test body"
mailto_link = email_handl.get_mailto_link(to = email_address, subject = subject, body = body)
mailto_link

'mailto:mr.mehmet@example.com?subject=test&body=test%20body'

In [31]:
eh = EmailHandler()
eh.get_todays_emails()

[{'subject': '=?UTF-8?Q?Aktif_olun:_10_adet/grup_LED_sopa_el?=\r\n =?UTF-8?Q?ektronik_=C4=B1=C5=9F=C4=B1k_sopa_klip_=C3=A7ubuk_uc..._?=',
  'to': 'maho0aga@hotmail.com',
  'from': 'AliExpress <ae-notice04.a3@deals.aliexpress.com>',
  'date': 'Thu, 8 Aug 2024 18:52:54 -0700 (PDT)',
  'body': '<meta http-equiv="Content-Type" content="text/html; charset=utf-8"><div style="display:none"><img style="display:none" src="http://ae.mmstat.com/ae.edm.edm_open?tracelog=rowan-ae_usertouch-aeug_edm_41192_1_tr_TR-2024-08-08&amp;rowan_msg_id=mahoCOWS_41192_$35f3de42282c406d81fe029d5ae18aed"></div><br><!--aeug_edm_41192#41192--><!doctype html>\r\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office">\r\n <head> \r\n  <meta name="viewport" content="width=device-width, initial-scale=1"> \r\n  <!--[if !mso]><!----> \r\n  <meta http-equiv="X-UA-Compatible" content="IE=edge"> \r\n  <!--<![endif]--> \r\n   \r\n  <style type="text

# Image Generator

In [33]:
class Size:
  """Image size class."""
  height: int
  width: int
  def __init__(self, height, width):
    self.width = width
    self.height = height

class ImageSize(Enum):
  """Image size enumeration."""
  #heightxwidth
  STANDARD = Size(1024, 1024)
  WIDE = Size(640, 1536)
  LONG = Size(1536, 640)

class ImageGenerator:
  """Generates images using the Stable Diffusion API."""
  engine_id: str = None
  api_host: str = None
  api_key: str = None

  def __init__(self, api_host: str = None, api_key: str = None, engine_id:str = "stable-diffusion-v1-6"):
    self.engine_id = engine_id
    self.api_host = api_host if api_host else os.getenv("API_HOST", "https://api.stability.ai")
    self.api_key = api_key if api_key else os.getenv("STABLE_API_KEY")
    if self.api_key is None:
      raise Exception("Missing Stability API key.")

  def invoke(self,
      prompt: str,
      scale: int = 7,
      steps: int = 30,
      size: ImageSize = ImageSize.STANDARD,
      custom_size: Size = None,
      samples: int = 1, path = "./sd_generated_images"):
    """Generates images using the Stable Diffusion API."""
    image_size = size.value
    if custom_size:
      image_size = custom_size

    response = requests.post(
    f"{self.api_host}/v1/generation/{self.engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        },
        json={
            "text_prompts": [
                {
                    "text": f"{prompt}"
                }
            ],
            "cfg_scale": scale,
            "height": image_size.height,
            "width": image_size.width,
            "samples": samples,
            "steps": steps,
        },
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    data = response.json()
    if not os.path.exists(path):
        os.makedirs(path)
    for i, image in enumerate(data["artifacts"]):
        photo_id = str(uuid.uuid1())
        with open(f"{path}/v1_txt2img_{i}_id[{photo_id}].png", "wb") as f:
            f.write(base64.b64decode(image["base64"]))
    return f"One image has been succestfully generated with prompt: {prompt}", f"{path}/v1_txt2img_{i}_id[{photo_id}].png"



# Retriever

In [34]:
import chromadb
chroma_client = chromadb.Client()

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="my_collection")

# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "ASELSAN is a leading defense company in Turkey, known for its expertise in designing and developing advanced radar systems. Their technology is critical for military surveillance and air defense. ASELSAN’s radar systems are used in various platforms, including aircraft and naval vessels.",
        "ASELSAN is at the forefront of communication technology, providing sophisticated solutions for secure and reliable communication. Their systems are deployed in both military and civilian applications, ensuring robust connectivity in challenging environments. ASELSAN’s communication solutions include secure satellite communication and tactical radio systems.",
        "ASELSAN plays a pivotal role in electronic warfare with advanced systems designed to counteract enemy electronic attacks. Their electronic warfare solutions include jamming systems and electronic countermeasures that protect military assets from various threats. ASELSAN’s technology is integral to maintaining operational superiority in electronic warfare.",
        "ASELSAN produces a wide range of high-tech military avionics, including navigation systems and flight control systems for various aircraft. Their avionics are designed to enhance the operational capabilities and safety of military aircraft. ASELSAN's innovations in avionics support both manned and unmanned aerial vehicles.",
        "ASELSAN constructs its Sustainability Strategy with a holistic approach, based on the principles of “SAFEGUARDING THE FUTURE” and in line with the UN Sustainability Goals. In this context; ASELSAN motivates both its own processes, the ecosystem in which it is involved, and all its stakeholders, and encourages them to work within the framework of the same plan. ASELSAN; It aims to leave a livable world to future generations and to develop this world in a positive way, with active participation and voluntary cooperation in particular on the themes of society, economy and the world.",
        """For ASELSAN, providing information security is crucial. ASELSAN, with this purpose, guarantees to
        protect all information assets it has, to provide the conditions of information security within the frame
        of international standards, laws and legislations, to improve, enhance and consider the information
        security constantly by managing present and potential risks that is likely to occur. The importance of
        the concept of cyber security has increased with the technological developments and digitalization.
        ASELSAN determines information security requirements and takes precautions against cyber security
        threats in line with its information security policy.""",
        """ASELSAN designs systems suitable for the layered air defense concept, in which Air Defense Weapons and Missile Systems are integrated with high performance sensor, command control, communication and fire control systems.
        ASELSAN added Air Defense Early Warning and Command Control System (HERİKKS) and Air Defense Radar products to its Air Defense System solutions, which started with the Stinger Joint Production Program about 30 years ago and continued with the Pedestal Mounted Stinger (KMS) project. In this sense, ASELSAN has achieved the success of providing solutions in this field to the armies of the world, especially the Turkish Armed Forces, by producing complete systems that include the three cornerstones of the Air Defense System, Weapons, Sensors and Command Control.
        ASELSAN, with its 25 years of experience in air defense, can use various caliber weapons, launch missiles and/or rockets within the scope of air defense projects such as KORKUT Self-Propelled Air Defense Weapon System, HİSAR Low/Medium Altitude Air Defense Missile System, Long Range Area Air and Missile Defense Systems. continues the design, development and production of various air defense weapon systems that can be integrated into land, sea and air platforms. It also carries out the modernization of the air defense systems available in the Armed Forces' inventory in line with current needs.
        """,

    ],
    ids=["aselsan_radar", "aselsan_communications", "aselsan_electronic_warfare", "aselsan_avionics", "aselsan_sustainability", "aselsan_information_security_policy", "aselsan_air_defence_technologies"]
)


class Retr:
  def __init__(self):
    pass
  def get_relevant_documents(self, query):
      logger.log(f"Rett -> query: {query}")
      results = collection.query(
      query_texts=[query], # Chroma will embed this for you
      n_results=2 # how many results to return
      )
      ret_value = "-"*50+"\n"
      for i in range(2):
        id = results["ids"][0][i]
        document = results["documents"][0][i]
        res = f"Document: **{id}**\n\n{document}"
        ret_value += res + "\n"+"-"*50+"\n"

      return ret_value




In [ ]:
rrr = Retr()
rrr.get_relevant_documents("Give me some information about sustainability strategies")

'--------------------------------------------------\nDocument: aselsan_sustainability\n\nASELSAN constructs its Sustainability Strategy with a holistic approach, based on the principles of “SAFEGUARDING THE FUTURE” and in line with the UN Sustainability Goals. In this context; ASELSAN motivates both its own processes, the ecosystem in which it is involved, and all its stakeholders, and encourages them to work within the framework of the same plan. ASELSAN; It aims to leave a livable world to future generations and to develop this world in a positive way, with active participation and voluntary cooperation in particular on the themes of society, economy and the world.\n--------------------------------------------------\nDocument: aselsan_information_security_policy\n\nFor ASELSAN, providing information security is crucial. ASELSAN, with this purpose, guarantees to\n        protect all information assets it has, to provide the conditions of information security within the frame\n        

In [ ]:
results = collection.query(
    query_texts=["Give me some information about air defence technologies in Aselsan"], # Chroma will embed this for you
    n_results=2 # how many results to return
)

ret_value = "-"*50+"\n"
for i in range(2):
  id = results["ids"][0][i]
  document = results["documents"][0][i]
  res = f"Document: {id}\n\n{document}"
  ret_value += res + "\n"+"-"*50+"\n"

print(ret_value)





--------------------------------------------------
Document: aselsan_air_defence_technologies

ASELSAN designs systems suitable for the layered air defense concept, in which Air Defense Weapons and Missile Systems are integrated with high performance sensor, command control, communication and fire control systems.
        ASELSAN added Air Defense Early Warning and Command Control System (HERİKKS) and Air Defense Radar products to its Air Defense System solutions, which started with the Stinger Joint Production Program about 30 years ago and continued with the Pedestal Mounted Stinger (KMS) project. In this sense, ASELSAN has achieved the success of providing solutions in this field to the armies of the world, especially the Turkish Armed Forces, by producing complete systems that include the three cornerstones of the Air Defense System, Weapons, Sensors and Command Control.
        ASELSAN, with its 25 years of experience in air defense, can use various caliber weapons, launch missile

In [ ]:
os.system("rm -r ./db")
os.system("rm -r ./data")

books = {
    #"odyssey": "https://www.gutenberg.org/cache/epub/1727/pg1727.txt",
    "romeo_and_juliet": "https://www.gutenberg.org/cache/epub/1513/pg1513.txt",
    #"frankenstein": "https://www.gutenberg.org/cache/epub/84/pg84.txt",
    # "les_miserables": "https://www.gutenberg.org/cache/epub/135/pg135.txt"
}

directory_path = "./data"

os.makedirs(directory_path, exist_ok=True)
print(f"Directory '{directory_path}' created successfully.")

for book_name, book_url in books.items():
    print(f"Downloading {book_name}...")
    response = requests.get(book_url)
    if response.status_code == 200:
        with open(os.path.join(directory_path, f"{book_name}.txt"), "wb") as file:
            file.write(response.content)
    else:
        print(f"Failed to download {book_name}. Status code: {response.status_code}")

print("Book installation finished!")

if not os.path.exists("./db"):
    os.makedirs("./db")
    print("Directory './db' created successfully.")

Directory './data' created successfully.
Book installation finished!
Directory './db' created successfully.


In [ ]:

import os
from datetime import datetime, timedelta, timezone
from typing import List, Any

# Placeholder imports for missing classes (replace with actual imports)
# from some_module import CohereEmbeddings, RecursiveCharacterTextSplitter, Chroma, TextLoader
class Embedder:
    """Embeds text using a specified model on a given platform."""

    platform_dict = {
        "cohere": CohereEmbeddings
    }

    def __init__(self, platform: str, model: str):
        if platform not in self.platform_dict:
            raise ValueError(f"Platform {platform} is not supported.")

        self.model_name = model
        self.platform = platform
        self.model = self.platform_dict[self.platform](model=self.model_name)

class Chunker:
    """Chunks text into smaller pieces with specified overlap."""

    def __init__(self, chunk_size: int, chunk_overlap: int):
        """
        Initializes the Chunker with size and overlap.

        chunk_size: The number of characters in each chunk.
        chunk_overlap: The number of characters to overlap between chunks.
        """
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.chunker = RecursiveCharacterTextSplitter(chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)

    def get_chunked_documents(self, documents: list) -> list:
        """Splits documents into chunks."""
        return self.chunker.split_documents(documents)

class VectorStore:
    """Stores vectors and manages persistence."""

    def __init__(self, docs: List[Any], embedder: Any, persist_directory: str = "./vector_database/chroma_db"):
        self.docs = docs
        self.embedder = embedder
        self.persist_directory = persist_directory
        self.set_environment()
        self.store = Chroma(persist_directory=persist_directory, embedding_function=self.embedder)

    def set_environment(self) -> None:
        """Sets up the environment by ensuring directory exists and populates initial data."""
        if not os.path.exists(self.persist_directory):
            os.makedirs(self.persist_directory)
        if len(os.listdir(self.persist_directory)) == 0:
            try:
                Chroma.from_documents(self.docs, self.embedder, persist_directory=self.persist_directory)
            except Exception as e:
                raise RuntimeError(f"Failed to create initial Chroma store: {e}")

    def add_new_document(self, documents: List[Any]) -> int:
        """Adds new documents to the store and saves."""
        for document in documents:
            self.docs.append(document)
            try:
                new_embedding = self.embedder.embed_query(document)
                self.add_documents([document], [new_embedding])
            except Exception as e:
                raise RuntimeError(f"Failed to add document: {e}")
        self.save()
        return 1

class Retriever:
    """Retrieves documents from the vector store based on queries."""

    def __init__(self):
        self.current_dir = os.getcwd()
        self.source_dir = os.path.join(self.current_dir, "data")
        self.db_dir = os.path.join(self.current_dir, "db")
        self.persistent_directory = os.path.join(self.db_dir, "chroma_db")

        self.source_files = [file for file in os.listdir(self.source_dir) if file.endswith(".txt")]
        self.documents = []

        for source_file in self.source_files:
            file_path = os.path.join(self.source_dir, source_file)
            loader = TextLoader(file_path)
            file_docs = loader.load()
            for doc in file_docs:
                doc.metadata["source"] = source_file
                self.documents.append(doc)

        self.embedder = Embedder(platform="cohere", model="embed-english-light-v3.0").model
        chunker = Chunker(chunk_size=1000, chunk_overlap=500)
        chunked_documents = chunker.get_chunked_documents(self.documents)

        # Instantiate VectorStore
        self.db = VectorStore(chunked_documents, self.embedder, self.persistent_directory).store

    def get_relevant_documents(self, _query: str) -> str:
        """Retrieves relevant documents based on the query."""
        query = _query
        relevant_docs = self.db.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 2}
        ).invoke(query)

        num_of_docs = len(relevant_docs)

        if num_of_docs == 0:
            return "No relevant documents found."

        final_answer = f"{num_of_docs} relevant documents found for your query '{query}':\n"
        for i, doc in enumerate(relevant_docs, 1):
            final_answer += f"Document {i}:\n{doc.page_content}\n"
            final_answer += f"Source {doc.metadata['source']}\n"

        return final_answer



In [ ]:
r = Retriever()

RuntimeError: Failed to create initial Chroma store: could not convert string to float: 'float_'

# MySQL


In [35]:
import sqlite3

class SqlHandler:
    def __init__(self, db_name='demo.db'):
        self.db_name = db_name
        self.connection = sqlite3.connect(self.db_name, check_same_thread=False)
        self.cursor = self.connection.cursor()

    def get_column_information(self, table):
        # Get column names and their datatypes
        query = f"PRAGMA table_info({table});"
        self.cursor.execute(query)
        columns_info = self.cursor.fetchall()
        column_details = {column[1]: column[2] for column in columns_info}  # {column_name: datatype}
        return column_details

    def fetch_result(self, query):
        # Execute a query and return the results
        self.cursor.execute(query)
        results = self.cursor.fetchall()
        return results

    def populate(self):
        # Create and populate the 'spendings' table
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS spendings (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                category TEXT NOT NULL,
                amount REAL NOT NULL,
                date TEXT NOT NULL
            )
        ''')

        # Clear the table before populating it with new data
        self.cursor.execute('DELETE FROM spendings')

        # Insert sample data
        self.cursor.executemany('''
            INSERT INTO spendings (category, amount, date) VALUES (?, ?, ?)
        ''', [
            ('Food', 50.25, '2024-08-01'),
            ('Transport', 15.75, '2024-08-02'),
            ('Entertainment', 75.00, '2024-08-03'),
            ('Utilities', 120.50, '2024-08-04')
        ])
        self.connection.commit()

    def __del__(self):
        # Close the connection when the object is destroyed
        self.connection.close()

# Example usage:

sql_handler2 = SqlHandler()  # This will use 'demo.db'

# Populate the database with data
sql_handler2.populate()

# Get column information
columns = sql_handler2.get_column_information(table = 'spendings')
print("Column Information:", columns)

# Fetch results
results = sql_handler2.fetch_result('SELECT * FROM spendings')
print("Results:", results)


Column Information: {'id': 'INTEGER', 'category': 'TEXT', 'amount': 'REAL', 'date': 'TEXT'}
Results: [(5, 'Food', 50.25, '2024-08-01'), (6, 'Transport', 15.75, '2024-08-02'), (7, 'Entertainment', 75.0, '2024-08-03'), (8, 'Utilities', 120.5, '2024-08-04')]


# BasilGPT

In [36]:
cur = None

prompt = hub.pull("hwchase17/structured-chat-agent")
logger = Logger("logs")
balance: float = 100

#retriver = Retriever()
retriver = Retr()
email_handler = EmailHandler()
db_handler = SqlHandler()

last_image_set = None
# Define function schemas for structured tools
class NoArgs(BaseModel):
    pass

class BalanceArgs(BaseModel):
    amount: float

class SearchArgs(BaseModel):
    query: str

class DataArgs(BaseModel):
    query: str

class SetDatabaseArgs(BaseModel):
    database_name: str
    user_name: str
    password: str

class ForecastWeatherArgs(BaseModel):
    city: str
    forecast_days: int
    hour: int

class CurrentWeahterArgs(BaseModel):
  city: str

class CurrencyExchangeRateArgs(BaseModel):
  from_currency: str
  to_currency: str

class ImageArgs(BaseModel):
  prompt: str
  height: int
  width: int

class GetRelevantDocuments(BaseModel):
  query: str

class GenerateFileArgs(BaseModel):
  file_name: str
  file_content: str

class MailArgs(BaseModel):
  email_address: str
  subject: str
  body: str

def get_connection():
    global cur, db_name, db_password, db_user_name
    logger.log("get_connection")
    try:
        mydb = mysql.connector.connect(
            host="localhost",
            user='root',
            password='root',
            database='demo'
        )
        cur = mydb.cursor()
        return cur
    except mysql.connector.Error as err:
        logger.log(f"MySQL Connection Error: {err}")
        return None

def get_current_time():
    logger.log("get_current_time")
    """Returns the current time in HH:MM AM/PM format adjusted for Turkey (UTC+3)."""
    current_date_time = datetime.datetime.now()
    timezone_offset = datetime.timedelta(hours=3)
    turkey_time = current_date_time + timezone_offset
    return turkey_time.strftime("%I:%M %p")

def get_current_balance() -> str:
    """Returns the current balance of the user in terms of dollars ($)."""
    logger.log("get_current_balance")
    global balance
    return f"{balance}$ in the account"

def add_balance(amount: float) -> float:
    """Adds the given amount to the user's balance, then returns the recent balance in terms of dollars ($)."""
    logger.log(f"add_balance({amount})")
    global balance
    balance += float(amount)
    return balance

def search_wiki(query: str) -> str:
    """Returns the search result from wikipedia.com."""
    logger.log(f"search_wiki({query})")
    from wikipedia import summary
    try:
        return summary(query, sentences=2)
    except:
        return f"No results found for {query}"

def search_web(query: str) -> str:
    """Returns the search result from Google with GoogleSearchAPIWrapper().run(query) method.
    Never run this method unless you are told to use 'google search' tool.
    """
    logger.log(f"search_web({query})")
    search = GoogleSearchAPIWrapper()
    try:
      return search.run(query)
    except:
      return f"No results found for {query}"

def chat(message: str) -> str:
    """Placeholder chat function."""
    return message

def run_database_query(query: str) -> list:
    """Returns the result of the database query output."""
    return db_handler.fetch_result(query)


def get_database_information() -> dict:
    return db_handler.get_column_information(table="spendings")


def get_forecast_weather_condition(city:str, forecast_days:int, hour:int):
  """Returns a dictionary of forecast weather conditions of given city name."""
  logger.log(f"get_forecast_weather_condition(city:{city}, forecast_days:{forecast_days}, hour:{hour})")
  key = os.environ["WEATHER_API_KEY"]
  parameters = {
    "key": key,
    "q": city,
    "days": forecast_days,
    "hour": hour
  }
  response = requests.get(url="https://api.weatherapi.com/v1/forecast.json", params = parameters)

  if response.status_code == 200:
    data = response.json()
  else:
    return "Connection Failed"
  return data

def get_current_weather_condition(city:str):
  """Returns a dictionary of current weather conditions of given city name."""
  logger.log(f"get_current_weather_condition(city:{city})")
  key = os.environ["WEATHER_API_KEY"]
  parameters = {
    "key": key,
    "q": city,
  }
  response = requests.get(url="https://api.weatherapi.com/v1/current.json", params = parameters)
  if response.status_code == 200:
    data = response.json()
  else:
    return "Connection Failed"
  return data

def get_currency_exchange_rate(from_currency:str, to_currency:str)->dict:
  """
  Retrieves the current exchange rate from Alpha Vantage API for a given currency pair.

  Args:
  - from_currency (str): The base currency code.
  - to_currency (str): The target currency code.

  Returns:
  - dict: A dictionary containing the exchange rate details, including 'from_currency',
          'to_currency', 'exchange_rate', and other relevant information.
  """
  logger.log(f"get_currency_exchange_rate(from_currency:{from_currency}, to_currency:{to_currency})")
  key = os.environ["AV_EXCH_API_KEY"],
  parameters = {
    "apikey": key,
    "from_currency": from_currency,
    "to_currency": to_currency,
    "function": "CURRENCY_EXCHANGE_RATE"
  }
  final_url = requests.Request('GET', "https://www.alphavantage.co/query", params=parameters).prepare().url
  print(final_url)
  response = requests.get(url="https://www.alphavantage.co/query", params = parameters)

  if response.status_code == 200:
    data = response.json()
    return data
  else:
    return {"error": "Could not get data"}

def generate_image(prompt: str, height: int, width: int) -> str:
  """
  Generates an image based on a given prompt using the Stable Diffusion API.

  Args:
  - prompt (str): The text prompt or description for generating the image.
  - height (int): The desired height of the generated image in pixels.
  - width (int): The desired width of the generated image in pixels.

  Returns:
  - success_message (str)
  """
  logger.log(f"generate_image(prompt:{prompt}, height:{height}, width:{width})")
  size = Size(height, width)
  responses = ["is in your presents..", "is generated.", "is generated successfully."]
  try:
    response, image_path = ImageGenerator().invoke(prompt, custom_size= size, steps = 50)
    global last_image_set
    last_image_set = image_path
    logger.log(f"image_response: {response}")
  except:
    return "Connection Failed"
  return f"An image of **{prompt}** "+random.choice(responses)


def get_relevant_documents_from_sources(query: str)->str:
  """Returns the most relevant documents from the provided sources."""
  logger.log(f"get_relevant_documents_from_sources(query:{query})")
  return retriver.get_relevant_documents(query)

def get_infromation(query: str)->str:
  """
  Retrieves information based on the given query by first checking for relevant documents from predefined sources.

  The function performs the following steps:
  1. Attempts to retrieve relevant documents related to the query from a set of predefined sources.
  2. If no relevant documents are found (i.e., the result is an empty list), it then performs a search on Wikipedia to find information related to the query.
  3. Returns either the relevant documents or the result from the Wikipedia search, depending on the availability of information.

  Args:
    query (str): The search query or question for which information is being requested.

  Returns:
    str: The relevant documents if available; otherwise, the result from the Wikipedia search.
  """
  logger.log(f"get_infromation(query:{query})")
  relevant_documents = get_relevant_documents_from_sources(query)
  if len(relevant_documents) == 0:
    return search_wiki(query)
  return relevant_documents

def generate_file(file_name: str, file_content: str) -> str:
  """
  Creates a file in the 'generated_files' directory with the specified name and content.

  If a file with the specified name already exists in the 'generated_files' directory,
  a unique file name will be generated by appending a numeric suffix to avoid overwriting.

  Args:
      file_name (str): The name of the file to be created.
      file_content (str): The content to write into the file.

  Returns:
      str: A message indicating the path where the file was created.

  Examples:
        >>> generate_file('test_file.txt', 'This is a test.')
        'The file was successfully created at: /current/working/directory/generated_files/test_file.txt'

        >>> # Test case where the file already exists
        >>> generate_file('test_file.txt', 'This is another test.')
        'The file was successfully created at: /current/working/directory/generated_files/test_file_1.txt'

        >>> # Creating a Python file with specific content
        >>> python_code = "print('Hello, World!')"
        >>> generate_file('hello_world.py', python_code)
        'The file was successfully created at: /current/working/directory/generated_files/hello_world.py'

        >>> # Handling case where the Python file already exists
        >>> generate_file('hello_world.py', "print('Goodbye, World!')")
        'The file was successfully created at: /current/working/directory/generated_files/hello_world_1.py'
  """
  logger.log(f"generate_file(file_name:{file_name}, file_content:{file_content})")
  target_directory = os.path.join(os.getcwd(), "generated_files")
  if not os.path.exists(target_directory):
    os.mkdir(target_directory)
  file_path = os.path.join(target_directory, file_name)
  if os.path.exists(file_path):
    file_path = os.path.join(target_directory, generate_file_hepler(target_directory, file_name))
  try:
    with open(file_path, "w") as file:
      file.write(file_content)
  except:
    return "File could not created"
  return f"The file gereated under the path: {file_path}"

def generate_file_hepler(target_directory: str, file_name: str)->str:
  """Generates a file with the given name and content."""
  logger.log(f"generate_file_hepler(file_name:{file_name})")
  for i in range(2, 2**32):
    if os.path.exists(os.path.join(target_directory, f"cp{i}_{file_name}")):
      continue
    else:
      return f"cp{i}_{file_name}"

def write_email(email_address: str, subject: str = '', body: str = '') -> str:
    """
    Generates a formatted email message and a mailto link for sending the email.

    This function constructs a `mailto` link using the provided email address, subject, and body.
    It also generates a formatted string that includes the recipient's email address, the subject,
    and the body of the email. Additionally, it provides a Markdown-formatted link that, when clicked,
    will open the user's default email client with the composed email.

    Args:
        email_address (str): The recipient's email address.
        subject (str, optional): The subject of the email. Defaults to an empty string.
        body (str, optional): The body content of the email. Defaults to an empty string.

    Returns:
        str: A formatted string with the recipient's email address, the subject, the body, and a Markdown
             link to view and send the email via the `mailto` link.
    """
    logger.log(f"send_mail(email_address:{email_address}, subject:{subject}, body:{body})")
    mailto_link = email_handler.get_mailto_link(to = email_address, subject = subject, body = body)
    mardown_link = f" [**Click here**]({mailto_link}) to view and send the email."
    return_value = f"This is the email I generated for you\nTo:{email_address}\n**{subject}**\n````\n{body}\n````\n{mardown_link}"
    return return_value

def get_todays_emails() -> list:
    """
    Retrieves and returns a list of today's emails.

    This function fetches emails received today using the `email_handler` object, reverses the order of the emails so that the most recent emails appear first, and then returns the list. Each email in the returned list is represented as a dictionary containing the following details:

    - 'subject': The subject line of the email.
    - 'to': The recipient's email address.
    - 'from': The sender's email address.
    - 'date': The date and time when the email was received, in GMT.
    - 'body': The content of the email.

    The list is formatted so that the newest emails are at the beginning of the list.
    """
    global email_handler
    try:
      logger.log("read_todays_emails")
      todays_emails = email_handler.get_todays_emails()
      todays_emails.reverse()
    except Error as e:
      logger.log(f"Error: {e}")
      return []
    return todays_emails

tools = [
    StructuredTool(
        name="Current Time",
        func=get_current_time,
        description="Useful for when you need to know the current time",
        args_schema=NoArgs
    ),
    #StructuredTool(
    #    name="Search Wiki",
    #    func=search_wiki,
    #    description="Useful for when you need to know information about a topic",
    #    args_schema=SearchArgs,
    #),
    StructuredTool(
        name="Current Balance",
        func=get_current_balance,
        description="Useful for when you need to know the current balance in dollars",
        args_schema=NoArgs,
        max_execution_time=10
    ),
    StructuredTool(
        name="Add Balance",
        func=add_balance,
        description="Adds an amount to the user's balance",
        args_schema=BalanceArgs,
    ),
    StructuredTool(
        name="Google Search",
        func=search_web,
        description="Search Google for recent results.",
        args_schema=SearchArgs,
        max_execution_time=15
    ),
    StructuredTool(
        name="Run Database Query",
        func=run_database_query,
        description="Useful for when you need to run queries on database.",
        args_schema=DataArgs
    ),
    StructuredTool(
        name="Get Database Information",
        func=get_database_information,
        description="Should be used after 'Set Database Information' tool is used. Returns the name of the tables, and their attributes names",
        args_schema=NoArgs
    ),
    StructuredTool(
        name = "Get Forecast Weather Condition",
        func = get_forecast_weather_condition,
        description = "Useful for when you need to know the current and forecast weather condition",
        args_schema = ForecastWeatherArgs,
        max_execution_time=10
    ),
    StructuredTool(
        name = "Get Current Weather Condition",
        func = get_current_weather_condition,
        description = "Useful for when you need to know the current weather condition",
        args_schema = CurrentWeahterArgs,
        max_execution_time=10
    ),
    StructuredTool(
        name = "Get Currency Exchange Rate",
        func = get_currency_exchange_rate,
        description = "Useful for when you need to know the current exchange rate",
        args_schema = CurrencyExchangeRateArgs,
        max_execution_time=15
    ),
    StructuredTool(
        name = "Generate Image",
        func = generate_image,
        description = "Useful for when you need to generate an image",
        args_schema = ImageArgs,
        return_direct=True
    ),
    #StructuredTool(
    #    name = "Get Relevant Documents From Sources",
    #    func = get_relevant_documents_from_sources,
    #    description = "Useful for when you need to get the most relevant documents from the provided sources",
    #    args_schema = GetRelevantDocuments,
    #    max_execution_time=10
    #),
    StructuredTool(
        name = "Get Information From Custom Sources",
        func = get_infromation,
        description = "Useful for getting information from sources.",
        args_schema = GetRelevantDocuments
    ),
    StructuredTool(
        name = "Send Mail",
        func = write_email,
        description = "Useful for when you write an email.",
        args_schema = MailArgs,
        return_direct=True
    ),
    StructuredTool(
        name = "Generate File",
        func = generate_file,
        description = "Useful for when you need to generate a file.",
        args_schema = GenerateFileArgs
    ),
    StructuredTool(
        name = "Read Today's Emails",
        func = get_todays_emails,
        description = "Useful for when you need to read today's emails.",
        args_schema = NoArgs
    ),
]
"""
StructuredTool(
    name="Chat",
    func=chat,
    description="Useful for when you need to chat",
    args_schema=SearchArgs  # Assuming chat function takes a string message
),
"""
llm = ChatCohere(model="command-r-plus")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

agent_executer = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=False,
    handle_parsing_errors=True,
    early_stopping_method="generate", #"force"

    )
suffix = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
"""
suffix = ""
initial_message = """
**System Initialization Message:**

Welcome to AIGENT-007, your multi talented AI assistant designed to provide helpful answers using available tools.

**ALWAYS** introduce yourself at the beginning of the conversation.
**ALWAYS** ask the user's name to communicate better.
**ALWAYS** wait for the tool's response. The tool will give you the answer, do not ask the question again.

### Database Tools Overview:
1. **Getting Database Information**: Retrieve database information from local variables.
2. **Running Queries**: Execute queries in the database and return the results.

### Guidelines for Database Interaction:
- **Verification of Database Information**: Always verify if necessary database information is available. Prompt the user to provide any missing information. Do not generate missing information.
- **Initial Database Setup**: At the beginning, search the entire database to gather table names and their attribute names using the `get_database_information` method.
- **Responding to Queries**: Answer the user's questions based on results obtained from the `run_database_query(query)` function.

### Guidelines for Weather Interaction:
- **Deciding the Day Parameter**: If the user asks about rain for tomorrow, the day parameter should be 2.
- **Assumption of Hour Parameter**: If the user does not specify the hour, default to 0 for the hour parameter.
- **Specifications of Day Parameter**: The day parameter is an integer representing the number of days in the forecast. For example, if the user asks about the weather tomorrow, the day parameter should be 2.
- **Specifications of Hour Parameter**: The hour parameter is an integer in 24-hour format. For instance, if the current time is 8 am, the hour parameter will be 8; if it's 8 pm, the hour parameter will be 20.

### Guidelines for Exchange Rate Interaction:
- **Assumption of Currency Parameters**: If the user does not specify currency codes (from_currency and to_currency), default to USD as the base currency and TRY as the target currency.
- **Currency Code Specification**: Currency codes should follow the ISO 4217 standard. For example, "USD" for US Dollar, "EUR" for Euro, "GBP" for British Pound Sterling, "TRY" for Turkish Lira.
- **Functionality**: Retrieve the current exchange rate from the Alpha Vantage API for a specified currency pair using the CURRENCY_EXCHANGE_RATE function.
- **API Key Requirement**: Ensure the Alpha Vantage API key (AV_EXCH_API_KEY) is securely stored and accessible as an environment variable.
- **Error Handling**: If the function fails to retrieve data due to network issues or API errors, return None and print an error message with the HTTP status code.

### Guidelines for Image Generation:
- **Image Size Specification**:
  - Default size: 1024 x 1024 if unspecified.
  - Specific sizes:
    - **Standard**: 1024 x 1024
    - **Wide**: 640 x 1536 for wallpapers, backgrounds, or wide banners.
    - **Long**: 1536 x 640 for social media or phone formats.

"""

def suffix_generator(query: str, search_tool: str):
  suffix = "These are the search results. Answer the question with these information only, do not use pre-trained knowladge or any other tools. If you find the answer, always mention about the source.\nThe provided information:\n"
  if search_tool == "automatic":
      return ""

  if search_tool == "google":
      suffix += "Source: Google\n" + search_web(query)
  elif search_tool == "wikipedia":
      suffix += "Source: Wikipedia\n"+search_wiki(query)
  elif search_tool == "sources":
      suffix += get_relevant_documents_from_sources(query)
  elif search_tool == "wikipedia xor sources":
      suffix += get_infromation(query)

  suffix += f"\n\nThis is the question you should answer:\n{query}"
  logger.log(f"Generated suffic for {search_tool} is: {suffix}")
  return suffix



memory.chat_memory.add_message(SystemMessage(content=initial_message))

history = []
def chat(user_input, search_tool):
  try:
    inp = user_input
    if search_tool != "automatic":
      user_input = suffix_generator(user_input, search_tool)
    memory.chat_memory.add_message(HumanMessage(content=user_input+suffix))
    response = agent_executer.invoke({"input": user_input+suffix})
    memory.chat_memory.add_message(AIMessage(content=response["output"]))
    logger.log(f"output:\n\n{response['output']}\n")
    history.append(["You: "+inp, "Aigent: "+response["output"]])
    return history, last_image_set
  except Exception as e:
    logger.log(f"Error: {e}")
    return e, None

interface = gr.Interface(
    fn=chat,
    inputs=[
        gr.Textbox(label="Query", value = ""),
        gr.Radio(
            choices=["automatic", "google", "wikipedia", "sources", "wikipedia xor sources"],
            value="automatic",
            label="Search Tool",
            info="Choose your search model\nIf you choose other than 'automatic', it will strictly search your checked tools."
        )
    ],
    outputs=[
        gr.Chatbot(label="Chat history",min_width=400, height=300),  # Automatically displays the history in a chat format
        gr.Image(label="Image")  # Remove this if not used
    ],
    examples=[
        ["What is the weather like in Ankara?", "automatic"],
        ["Write an email to Mr. Mehmet and ask for a meeting.", "automatic"],
        ["Would you like to check today's inbox and summarize please?", "automatic"],
        ["Write a python code that prints random numbers, and save the code.", "automatic"],
        ["Can you generate an image of Tank in a city?", "automatic"],
        ["What happened in 1945?", "wikipedia"],
        ["How sleeping sickness occure?", "sources"],

    ],
    title = "AIGENT-007",
    description = "Your multi-talented AI assistant.",
    article = "Hello! I am AIGENT-007, your multi-talented AI assistant. I can provide you with helpful answers and assist with various tasks. How can I help you today?"
)


interface.launch()

"""
You should use a clear and concise prompt structure when providing system messages to an LLM modal about using tools. This helps the LLM understand and follow your instructions effectively. Here's a suggested template:

"Reminder: Always utilize the provided tools to respond to user queries. You have access to various tools, including database interaction, weather updates, currency exchange rates, image generation, and more. Remember to respond accurately and thoroughly." By providing clear and specific instructions, you ensure that the LLM has a better understanding of the expected behavior and can assist users more effectively. Clear prompts are beneficial for LLMs.


### Guidlines for 'Get Information From Custom Sources'
- **ALWAYS** try to answer all question with this tool.
"""


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://93ae7517436033d318.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


'\nYou should use a clear and concise prompt structure when providing system messages to an LLM modal about using tools. This helps the LLM understand and follow your instructions effectively. Here\'s a suggested template:\n\n"Reminder: Always utilize the provided tools to respond to user queries. You have access to various tools, including database interaction, weather updates, currency exchange rates, image generation, and more. Remember to respond accurately and thoroughly." By providing clear and specific instructions, you ensure that the LLM has a better understanding of the expected behavior and can assist users more effectively. Clear prompts are beneficial for LLMs.\n\n\n### Guidlines for \'Get Information From Custom Sources\'\n- **ALWAYS** try to answer all question with this tool.\n'

In [ ]:
search_web("Who found Aselsan?")

"Ş., is a Turkish defense corporation headquartered in Ankara, Turkey. Its main operating area is research, development and manufacture of advanced military\xa0... No articles found. Aselsan A.Ş. did not contribute to any primary research papers from Nature Index journals in the current 12 month window. Return to\xa0... Developing technology for half a century, ASELSAN is a leading international defense electronics company that offers solutions in a wide range of fields from\xa0... ❖ TAFF was established by the merger of Land, Navy and Air. Forces ... aselsan.ir@aselsan.com.tr. Web Site www.aselsan.com.tr/en/investor-relations. ASELSAN continues to offer the best solutions and develop groundbreaking technologies in many fields such as air defense, radar, electronic warfare, electro-\xa0... Ş. Mehmet Akif Ersoy Mah. İstiklal Marşı Cad. No:16. 06200 Yenimahalle,. Ankara. Telephone. +90 850 828 1 828. E-Mail aselsan.ir@aselsan.com.tr. Web Site www. Apr 19, 2023 ... According to the claim 

In [ ]:
import json

class RecursivePrompter:
  llm
  sub_prompts = []
  list_length = 0
  responses = []

  def __init__(self):
    llm = ChatCohere(model="command-r-plus")

  def set_sub_prompts(self, prompts: str) -> None:
    response = llm.invoke(
    f"""
      You are a prompt engineer. Your job is get the prompt and divide them into individual prompts.
      **ALWAYS** give the all prompts in json format and do not add noncode text
      The format should be like this:
      {{
        "prompt1": "this is prompt 1",
        "prompt2": "this is prompt 2"
      }}
      Prompt:
      {prompts}
    """).content

    data = json.loads(response)
    self.sub_prompts = list(data.values())
    self.list_length = len(self.sub_prompts)

  def get_sub_prompt(self):
    if len(self.sub_prompts) > 0:
      return True, self.sub_prompts.pop(0)
    else:
      return False, ""

  def add_response(self, response):
    self.responses.append(response)

  def get_final_answer(self):
    questions = ""
    for elem in self.responses:
      questions += elem + "\n"
    resp = llm.invoke(f"""
      You are a prompt engineer. Your job is to summarize the given text without losing its meaning and detail.
      **ALWAYS** give the final answer only
      **SUMMARIZE** the following text:
      {questions}
      """).content
    self.responses = []
    return resp





In [ ]:
rec = RecursivePrompter()
rec.set_sub_prompts("Where the Ankara is and who is michael jackson?")
cont = True
while cont:
  cont, query = rec.get_sub_prompt()
  if cont == False:
    break
  ll = ChatCohere(model = "command-r-plus")
  print(query)
  response = ll.invoke(query).content
  rec.add_response(response)
print(rec.get_final_answer())

Where is Ankara located?
Who is Michael Jackson?
Ankara, Turkey's capital, experiences a continental climate with hot, dry summers and cold, snowy winters. Spring and autumn are mild, with occasional rain and sunshine. The city, located in central Turkey, is a historical hub, previously known as Ancyra. It is renowned for its administrative significance and cultural attractions. Michael Jackson, the 'King of Pop,' was an influential American singer, songwriter, and dancer. His four-decade career included groundbreaking music, dance, and fashion innovations, alongside a highly publicized personal life. Jackson's early career began with The Jackson 5, and his solo success included iconic albums like *Off the Wall* and *Thriller*. His impact on music, race relations, and humanitarian causes left a lasting legacy.


In [ ]:
"""
The weather in Ankara, Turkey, is currently sunny with a temperature of 30.5ºC or 86.9ºF. The humidity is relatively low at 29%, and the wind speed is 15.1 kph.
It is currently 04:15 PM.
The current exchange rate for USD to TRY is 32.99, last refreshed at 1:15 PM UTC today.
I've added $30 to your balance as requested. Your new balance is $160.
"""

"""
{
  "prompt1": "Generate an image for my AI presentation.",
  "prompt2": "What time is it?",
  "prompt3": "What is the weather like in Ankara?",
  "prompt4": "Check my balance.",
  "prompt5": "What is the current TL/USD exchange rate?",
  "prompt6": "Add 30 dollars to my balance."
}
"""


llm = ChatCohere(model="command-r-plus")
resp = llm.invoke("""
You are a prompt engineer. Your job is get the prompt and divide them into individual prompts.
**ALWAYS** give the all prompts in json format and do not add noncode text
The format should be like this:
{
  "prompt1": "this is prompt 1",
  "prompt2": "this is prompt 2"
}
Prompt:

What is the weather like in Ankara""").content

print(resp)

print("""

The results are:
The weather in Ankara, Turkey, is currently sunny with a temperature of 30.5ºC or 86.9ºF. The humidity is relatively low at 29%, and the wind speed is 15.1 kph.
It is currently 04:15 PM.
The current exchange rate for USD to TRY is 32.99, last refreshed at 1:15 PM UTC today.
I've added $30 to your balance as requested. Your new balance is $160.

""")

resp = llm.invoke("""
You are a prompt engineer. Your job is to summarize the given text without losing its meaning and detail.
**ALWAYS** give the final answer only
**SUMMARIZE** the following text:
The weather in Ankara, Turkey, is currently sunny with a temperature of 30.5ºC or 86.9ºF. The humidity is relatively low at 29%, and the wind speed is 15.1 kph.
It is currently 04:15 PM.
The current exchange rate for USD to TRY is 32.99, last refreshed at 1:15 PM UTC today.
I've added $30 to your balance as requested. Your new balance is $160.
""")
print("Summarization: ",resp.content)

{
  "prompt1": "What is the weather like in Ankara?"
}


The results are:
The weather in Ankara, Turkey, is currently sunny with a temperature of 30.5ºC or 86.9ºF. The humidity is relatively low at 29%, and the wind speed is 15.1 kph.
It is currently 04:15 PM.
The current exchange rate for USD to TRY is 32.99, last refreshed at 1:15 PM UTC today.
I've added $30 to your balance as requested. Your new balance is $160.


Summarization:  The weather in Ankara, Turkey, is sunny and warm with a temperature of 30.5ºC and low humidity. The wind speed is moderate at 15.1 kph. Today, at 04:15 PM, the USD to TRY exchange rate is 1 USD to 32.99 TRY. Your balance has been updated with an additional $30, resulting in a new total of $160.


In [ ]:
import json

sub_queries = """
              {
                "prompt1": "Generate an image for my AI presentation.",
                "prompt2": "What time is it?",
                "prompt3": "What is the weather like in Ankara?",
                "prompt4": "Check my balance.",
                "prompt5": "What is the current TL/USD exchange rate?",
                "prompt6": "Add 30 dollars to my balance."
              }
              """

data = json.loads(sub_queries)

for _, value in data.items():
  print(value)

lste = list(data.values())
lste.pop(0)
lste.pop(0)

Generate an image for my AI presentation.
What time is it?
What is the weather like in Ankara?
Check my balance.
What is the current TL/USD exchange rate?
Add 30 dollars to my balance.


'What time is it?'

In [ ]:
result = llm = ChatCohere(model="command-r-plus").invoke("""
How did juliet die according to this text. Explain briefly with full details.

Document 1:
FRIAR LAWRENCE.
I will be brief, for my short date of breath
Is not so long as is a tedious tale.
Romeo, there dead, was husband to that Juliet,
And she, there dead, that Romeo’s faithful wife.
I married them; and their stol’n marriage day
Was Tybalt’s doomsday, whose untimely death
Banish’d the new-made bridegroom from this city;
For whom, and not for Tybalt, Juliet pin’d.
You, to remove that siege of grief from her,
Betroth’d, and would have married her perforce
To County Paris. Then comes she to me,
And with wild looks, bid me devise some means
To rid her from this second marriage,
Or in my cell there would she kill herself.
Then gave I her, so tutored by my art,
A sleeping potion, which so took effect
As I intended, for it wrought on her
The form of death. Meantime I writ to Romeo
That he should hither come as this dire night
To help to take her from her borrow’d grave,
Being the time the potion’s force should cease.
But he which bore my letter, Friar John,
Was stay’d by accident; and yesternight
Return’d my letter back. Then all alone
At the prefixed hour of her waking
Came I to take her from her kindred’s vault,
Meaning to keep her closely at my cell
Till I conveniently could send to Romeo.
But when I came, some minute ere the time
Of her awaking, here untimely lay
The noble Paris and true Romeo dead.
She wakes; and I entreated her come forth
And bear this work of heaven with patience.
But then a noise did scare me from the tomb;
And she, too desperate, would not go with me,
But, as it seems, did violence on herself.
All this I know; and to the marriage
Her Nurse is privy. And if ought in this
Miscarried by my fault, let my old life
Be sacrific’d, some hour before his time,
Unto the rigour of severest law.

Source romeo_and_juliet.txt

Document 2:
NURSE.
I saw the wound, I saw it with mine eyes,
God save the mark!—here on his manly breast.
A piteous corse, a bloody piteous corse;
Pale, pale as ashes, all bedaub’d in blood,
All in gore-blood. I swounded at the sight.

JULIET.
O, break, my heart. Poor bankrout, break at once.
To prison, eyes; ne’er look on liberty.
Vile earth to earth resign; end motion here,
And thou and Romeo press one heavy bier.

NURSE.
O Tybalt, Tybalt, the best friend I had.
O courteous Tybalt, honest gentleman!
That ever I should live to see thee dead.

JULIET.
What storm is this that blows so contrary?
Is Romeo slaughter’d and is Tybalt dead?
My dearest cousin, and my dearer lord?
Then dreadful trumpet sound the general doom,
For who is living, if those two are gone?

NURSE.
Tybalt is gone, and Romeo banished,
Romeo that kill’d him, he is banished.

JULIET.
O God! Did Romeo’s hand shed Tybalt’s blood?

NURSE.
It did, it did; alas the day, it did.

Source romeo_and_juliet.txt

Document 3:
NURSE.
Ah sir, ah sir, death’s the end of all.

ROMEO.
Spakest thou of Juliet? How is it with her?
Doth not she think me an old murderer,
Now I have stain’d the childhood of our joy
With blood remov’d but little from her own?
Where is she? And how doth she? And what says
My conceal’d lady to our cancell’d love?

NURSE.
O, she says nothing, sir, but weeps and weeps;
And now falls on her bed, and then starts up,
And Tybalt calls, and then on Romeo cries,
And then down falls again.

ROMEO.
As if that name,
Shot from the deadly level of a gun,
Did murder her, as that name’s cursed hand
Murder’d her kinsman. O, tell me, Friar, tell me,
In what vile part of this anatomy
Doth my name lodge? Tell me, that I may sack
The hateful mansion.

 [_Drawing his sword._]

Source romeo_and_juliet.txt
""")
print(result.content)

In the provided text, it is revealed that Juliet died by suicide. 

Friar Lawrence explains that he gave Juliet a sleeping potion to make her appear dead, in order to prevent her forced marriage to County Paris. He intended to send a letter to Romeo, informing him of the plan so that they could reunite once she woke up. However, the letter did not reach Romeo, and when Friar Lawrence arrived to take Juliet from her tomb, he found Romeo and Paris dead. Upon waking, Juliet, desperate and surrounded by death, chose to take her own life rather than face the grim reality.


# Database

In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password = "root"
)

cur = mydb.cursor()
def populate():
  cur.execute("CREATE DATABASE IF NOT EXISTS demo")
  cur.execute("USE kumhrs")
  file = open("./db/sql.txt", "r")
  sql = file.read()
  cur.execute(sql)

# populate()
cur.execute("USE kumhrs")
cur.execute("SELECT COUNT(*) FROM Person")
print(cur.fetchall())
"""
cur.execute(
SELECT COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Hospital';
)
print(cur.fetchall())

"""
try:
  results = {}
  cur.execute("SHOW TABLES")
  tables = cur.fetchall()
  print(f"tables: {tables}")
  for table in tables:
      cur.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table[0]}'")
      columns = cur.fetchall()
      print("columns:", columns)
      results[table[0]] = columns
  print(results)
  print(f"results: {results}")
except:
  {}

hi
[(56,)]
tables: [('Appointment',), ('Department',), ('Doctor',), ('Hospital',), ('MedicineDetail',), ('Patient',), ('Person',), ('Prescription',), ('PrescriptionMedicine',)]
columns: [('adate', 'date'), ('aid', 'int'), ('atime', 'time'), ('doctor_id', 'int'), ('patient_id', 'int'), ('status', 'varchar')]
columns: [('department_id', 'int'), ('dname', 'varchar'), ('hospital_id', 'int')]
columns: [('department_id', 'int'), ('doctor_id', 'int'), ('rid', 'int')]
columns: [('haddress', 'varchar'), ('hname', 'varchar'), ('hospital_id', 'int')]
columns: [('dosage', 'varchar'), ('medicine_name', 'varchar'), ('mid', 'int'), ('note', 'varchar'), ('quantity', 'int'), ('tpd', 'int'), ('tpu', 'int')]
columns: [('emergency_contact', 'varchar'), ('patient_id', 'int'), ('rid', 'int')]
columns: [('address', 'varchar'), ('bdate', 'date'), ('email', 'varchar'), ('gender', 'varchar'), ('insnumber', 'varchar'), ('name', 'varchar'), ('password', 'varchar'), ('pnumber', 'varchar'), ('rid', 'int')]
columns:

# Weather Test

In [ ]:
import requests
from google.colab import userdata

def get_weather_condition(city:str, forecast_days:str, hour:str):
  """Returns a dictionary of weather conditions of given city name."""
  parameters = {
    "key": userdata.get('weather_api'),
    "q": city,
    "days": forecast_days,
    "hour": hour
  }
  response = requests.get(url="https://api.weatherapi.com/v1/forecast.json", params = parameters)

  if response.status_code == 200:
    data = response.json()
    print(data)
  else:
    print(response.status_code)

get_weather_condition("istanbul", "3", "1)")

{'location': {'name': 'Istanbul', 'region': 'Istanbul', 'country': 'Turkey', 'lat': 41.02, 'lon': 28.96, 'tz_id': 'Europe/Istanbul', 'localtime_epoch': 1721883816, 'localtime': '2024-07-25 8:03'}, 'current': {'last_updated_epoch': 1721883600, 'last_updated': '2024-07-25 08:00', 'temp_c': 27.5, 'temp_f': 81.6, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 12.8, 'wind_kph': 20.5, 'wind_degree': 48, 'wind_dir': 'NE', 'pressure_mb': 1006.0, 'pressure_in': 29.69, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 54, 'cloud': 3, 'feelslike_c': 28.6, 'feelslike_f': 83.4, 'windchill_c': 27.5, 'windchill_f': 81.6, 'heatindex_c': 28.6, 'heatindex_f': 83.4, 'dewpoint_c': 17.5, 'dewpoint_f': 63.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 7.0, 'gust_mph': 15.0, 'gust_kph': 24.1}, 'forecast': {'forecastday': [{'date': '2024-07-25', 'date_epoch': 1721865600, 'day': {'maxtemp_c': 30.0, 'maxtemp_f': 86.0, 'mintemp_c': 24.4,

# Gradio Test

In [ ]:
!ip a
!pwd

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
13: eth0@if14: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:1c:00:0c brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.28.0.12/16 brd 172.28.255.255 scope global eth0
       valid_lft forever preferred_lft forever
/content


In [ ]:
import gradio as gr

# Global history to keep track of the chat
history = []

def chat(query: str, search_tool: str):
    # Append query to history
    history.append((query, f"Query: {query}, Search Tool: {search_tool}"))

    # Return the formatted history and an image (if any, else None)
    return history, None  # Assuming `None` for the image output

interface = gr.Interface(
    fn=chat,
    inputs=[
        gr.Textbox(label="Query", value = ""),
        gr.Radio(
            choices=["automatic", "google", "wikipedia", "sources"],
            value="automatic",
            label="Search Tool",
            info="Choose your search model\nIf you choose other than 'automatic', it will strictly search your checked tools."
        )
    ],
    outputs=[
        gr.Chatbot(label="Chat history",min_width=400, height=300),  # Automatically displays the history in a chat format
        gr.Image(label="Image")  # Remove this if not used
    ],
    examples=[
        ["What is the weather like in Ankara?", "automatic"]
    ]
)

interface.launch()

"""
 [Click here](mailto:ggg@gmail.com?subject=Meeting%20Request%20from%20Jane%20G.&body=Dear%20Mr.%20Mehmet,%0D%0A%0D%0AI%20hope%20this%20email%20finds%20you%20well.%20My%20name%20is%20Jane%20G.,%20and%20I%20am%20writing%20to%20request%20a%20meeting%20with%20you%20to%20discuss%20a%20potential%20collaboration%20opportunity.%20I%20believe%20that%20a%20face-to-face%20conversation%20would%20be%20the%20best%20way%20to%20explore%20this%20further.%0D%0A%0D%0AI%20am%20happy%20to%20accommodate%20your%20schedule%20and%20meet%20at%20your%20convenience.%20Please%20let%20me%20know%20your%20preferred%20date,%20time,%20and%20location%20for%20the%20meeting.%0D%0A%0D%0AThank%20you%20for%20your%20time%20and%20consideration.%20I%20look%20forward%20to%20the%20opportunity%20to%20meet%20with%20you%20and%20discuss%20this%20matter%20in%20more%20detail.%0D%0A%0D%0ABest%20regards,%0D%0AJane%20G.) to view the email.
"""



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ca9eb4251ea49b2c89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


'\n [Click here](mailto:ggg@gmail.com?subject=Meeting%20Request%20from%20Jane%20G.&body=Dear%20Mr.%20Mehmet,%0D%0A%0D%0AI%20hope%20this%20email%20finds%20you%20well.%20My%20name%20is%20Jane%20G.,%20and%20I%20am%20writing%20to%20request%20a%20meeting%20with%20you%20to%20discuss%20a%20potential%20collaboration%20opportunity.%20I%20believe%20that%20a%20face-to-face%20conversation%20would%20be%20the%20best%20way%20to%20explore%20this%20further.%0D%0A%0D%0AI%20am%20happy%20to%20accommodate%20your%20schedule%20and%20meet%20at%20your%20convenience.%20Please%20let%20me%20know%20your%20preferred%20date,%20time,%20and%20location%20for%20the%20meeting.%0D%0A%0D%0AThank%20you%20for%20your%20time%20and%20consideration.%20I%20look%20forward%20to%20the%20opportunity%20to%20meet%20with%20you%20and%20discuss%20this%20matter%20in%20more%20detail.%0D%0A%0D%0ABest%20regards,%0D%0AJane%20G.) to view the email.\n'

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = random.choice(["Here an image: <img src='https://photographylife.com/wp-content/uploads/2014/09/Nikon-D750-Image-Samples-2.jpg'>"])
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()




Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://80b83bc4edd8f77b7e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, storage
import os

# Path to your Firebase service account key file
cred_file = './fs_key.json'

# Initialize Firebase
cred = credentials.Certificate(cred_file)
app = firebase_admin.initialize_app(cred, {
    'storageBucket': '1:284663160516:web:e8d8a70ffa7684dbcd7391.appspot.com'  # Replace with your actual bucket name
})

# Initialize Firestore and Storage
db = firestore.client()
bucket = storage.bucket(app=app)

def upload_image_to_firestore(image_path, collection_name, document_name):
    # Upload image to Firebase Storage
    blob = bucket.blob(os.path.basename(image_path))
    try:
        blob.upload_from_filename(image_path)
        blob.make_public()  # Make the file publicly accessible

        # Get the URL of the uploaded image
        image_url = blob.public_url

        # Save image metadata to Firestore
        doc_ref = db.collection(collection_name).document(document_name)
        doc_ref.set({
            'image_url': image_url,
            'image_name': os.path.basename(image_path),
            'uploaded_at': firestore.SERVER_TIMESTAMP
        })

        print(f"Image uploaded and metadata saved. Image URL: {image_url}")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    # Path to the image file
    image_path = './sd_generated_images/example.png'
    # Firestore collection and document names
    collection_name = 'images'
    document_name = 'image1'

    upload_image_to_firestore(image_path, collection_name, document_name)




ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

# Exchange Test

In [ ]:
import requests

def get_exchange(from_currency, to_currency):
  """
  Retrieves the current exchange rate from Alpha Vantage API for a given currency pair.

  Args:
  - from_currency (str): The base currency code.
  - to_currency (str): The target currency code.

  Returns:
  - dict: A dictionary containing the exchange rate details, including 'from_currency',
          'to_currency', 'exchange_rate', and other relevant information.
  """
  key = os.environ["AV_EXCH_API_KEY"],
  parameters = {
    "apikey": key,
    "from_currency": from_currency,
    "to_currency": to_currency,
    "function": "CURRENCY_EXCHANGE_RATE"
  }
  final_url = requests.Request('GET', "https://www.alphavantage.co/query", params=parameters).prepare().url
  print(final_url)
  response = requests.get(url="https://www.alphavantage.co/query", params = parameters)

  if response.status_code == 200:
    data = response.json()
    print(data)
get_exchange("USD", "TRY")

https://www.alphavantage.co/query?apikey=84UQSWBN5ZIZU7RG&from_currency=USD&to_currency=TRY&function=CURRENCY_EXCHANGE_RATE
{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD', '2. From_Currency Name': 'United States Dollar', '3. To_Currency Code': 'TRY', '4. To_Currency Name': 'Turkish Lira', '5. Exchange Rate': '32.96160000', '6. Last Refreshed': '2024-07-25 06:39:02', '7. Time Zone': 'UTC', '8. Bid Price': '32.96076000', '9. Ask Price': '32.96226000'}}


# SDXL Test

In [ ]:
class Size:
  """Image size class."""
  height: int
  width: int
  def __init__(self, height, width):
    self.width = width
    self.height = height

class ImageSize(Enum):
  """Image size enumeration."""
  #heightxwidth
  STANDARD = Size(1024, 1024)
  WIDE = Size(640, 1536)
  LONG = Size(1536, 640)

class ImageGenerator:
  """Generates images using the Stable Diffusion API."""
  engine_id: str = None
  api_host: str = None
  api_key: str = None

  def __init__(self, api_host: str = None, api_key: str = None, engine_id:str = "stable-diffusion-v1-6"):
    self.engine_id = engine_id
    self.api_host = api_host if api_host else os.getenv("API_HOST", "https://api.stability.ai")
    self.api_key = api_key if api_key else os.getenv("STABLE_API_KEY")
    if self.api_key is None:
      raise Exception("Missing Stability API key.")

  def invoke(self,
      prompt: str,
      scale: int = 7,
      steps: int = 30,
      size: ImageSize = ImageSize.STANDARD,
      custom_size: Size = None,
      samples: int = 1, path = "./sd_generated_images"):
    """Generates images using the Stable Diffusion API."""
    image_size = size.value
    if custom_size:
      image_size = custom_size

    response = requests.post(
    f"{self.api_host}/v1/generation/{self.engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        },
        json={
            "text_prompts": [
                {
                    "text": f"{prompt}"
                }
            ],
            "cfg_scale": scale,
            "height": image_size.height,
            "width": image_size.width,
            "samples": samples,
            "steps": steps,
        },
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    data = response.json()
    if not os.path.exists(path):
        os.makedirs(path)
    for i, image in enumerate(data["artifacts"]):
        photo_id = str(uuid.uuid1())
        with open(f"{path}/v1_txt2img_{i}_id[{photo_id}].png", "wb") as f:
            f.write(base64.b64decode(image["base64"]))
    return f"{i+1} images created and saved to the path {path}"

ImageGenerator().invoke("An astronout sitting on throne of the universe.", size = ImageSize.LONG)



'1 images created and saved to the path ./sd_generated_images'

In [3]:
import datetime
def get_current_time():
    """Returns the current time in HH:MM AM/PM format adjusted for Turkey (UTC+3)."""
    current_date_time = datetime.datetime.now()
    timezone_offset = datetime.timedelta(hours=3)
    turkey_time = current_date_time + timezone_offset
    return turkey_time.strftime("%I:%M %p")

print(get_current_time())

01:50 PM


# Generate File Test

In [ ]:
def generate_file(file_name: str, file_content: str) -> str:
  """Generates a file with the given name and content."""
  logger.log(f"generate_file(file_name:{file_name}, file_content:{file_content})")
  target_directory = os.path.join(os.getcwd(), "generated_files")
  if not os.path.exists(target_directory):
    os.mkdir(target_directory)
  file_path = os.path.join(target_directory, file_name)

  if os.path.exists(file_path):
    file_path = os.path.join(target_directory, generate_file_hepler(target_directory, file_name))
  with open(file_path, "w") as file:
    file.write(file_content)
  return f"The file gereated under the path: {file_path}"

def generate_file_hepler(target_directory: str, file_name: str)->str:
  """Generates a file with the given name and content."""
  logger.log(f"generate_file_hepler(file_name:{file_name})")
  for i in range(2, 2**32):
    if os.path.exists(os.path.join(target_directory, f"cp{i}_{file_name}")):
      continue
    else:
      return f"cp{i}_{file_name}"


In [ ]:
generate_file("test.py", "print('hello')")

'The file gereated under the path: /content/generated_files/cp3_test.py'

# Send Mail Test

In [ ]:
import webbrowser

def open_mailto_link(email: str, subject: str = '', body: str = ''):
    """Opens the default mail client with a mailto link."""
    mailto_link = f"mailto:{email}?subject={subject}&body={body}"
    webbrowser.open(mailto_link)

# Example usage
email = "example@gmail.com"
subject = "Hello"
body = "This is a test email."
open_mailto_link(email, subject, body)



In [ ]:
from IPython.display import display, HTML

def send_mail(email_address: str, subject: str = '', body: str = ''):
    """
    Attempts to open the default mail client with a pre-filled email message using a mailto link.

    This function generates an HTML page with a meta-refresh tag that tries to redirect the user
    to their default mail client with the specified email address, subject, and body. If automatic
    redirection is not supported, the user will see a clickable link to manually open their email client.

    Args:
        email_address (str): The recipient's email address.
        subject (str, optional): The subject of the email. Defaults to an empty string.
        body (str, optional): The body content of the email. Defaults to an empty string.

    Returns:
        str: A message indicating that the email page will be opened soon or if the redirection is attempted.

    Examples:
        >>> send_mail('example@gmail.com', 'Meeting Request', 'Please find the meeting details attached.')
        'The email page will be opened soon automatically.'
    """
    mailto_link = f"mailto:{email}?subject={subject}&body={body}"
    html_content = f'''
    <html>
    <head>
        <meta http-equiv="refresh" content="0;url={mailto_link}" />
    </head>
    <body>
        <p>If you are not redirected automatically, <a href="{mailto_link}">click here</a> to open your email client.</p>
    </body>
    </html>
    '''
    display(HTML(html_content))
    return "The email page will be opened soon automatically." + html_content

# Example usage
email = "ggg@gmail.com"
subject = "Test Mail"
body = "This is a test email."
send_mail(email, subject, body)
5



5

In [ ]:
from IPython.display import display, HTML

# Define the URL you want to open
url = 'https://www.google.com'

# Create an HTML string with a meta refresh tag to redirect to the URL
html_content = f'<html><head><meta http-equiv="refresh" content="0;url={url}" /></head><body></body></html>'

# Display the HTML content
display(HTML(html_content))


In [ ]:
!pip install SendGrid
import sendgrid
from sendgrid.helpers.mail import Mail

def send_email_with_sendgrid(api_key: str, email_address: str, subject: str = '', body: str = ''):
    """Sends an email using SendGrid API."""
    sg = sendgrid.SendGridAPIClient(api_key)
    email = Mail(
        from_email='your_email@example.com',
        to_emails=email_address,
        subject=subject,
        plain_text_content=body
    )
    response = sg.send(email)
    return response.status_code, response.body, response.headers

# Example usage
api_key = 'your_sendgrid_api_key'
email_address = "example@gmail.com"
subject = "Hello"
body = "This is a test email."
status_code, response_body, response_headers = send_email_with_sendgrid(api_key, email_address, subject, body)
print(f"SendGrid Response: {status_code}")


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 3.8 MB/s eta 0:00:00
  Created wheel for starkbank-ecdsa: filename=starkbank_ecdsa-2.2.0-py3-none-any.whl size=15969 sha256=751cca32141dd274d8253a8ca1019925afd35fef75ac185bcb4086ef8b00afcd
  Stored in directory: /root/.cache/pip/wheels/66/eb/94/ac982effd341c8d49f0b7c8d5fe0ee6f77150f015dd18af1e7
Successfully built starkbank-ecdsa


UnauthorizedError: HTTP Error 401: Unauthorized

In [ ]:
!pip install pyinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 685.0/685.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.3/341.3 kB 17.1 MB/s eta 0:00:00


In [ ]:
!pip install cx_Freeze


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.1/931.1 kB 29.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
from urllib.parse import quote

def send_email(to, subject='', body='', cc='', bcc=''):
    """
    Opens the default email client with pre-filled fields.

    Parameters:
        to (str): Recipient email address.
        subject (str): Subject of the email.
        body (str): Body of the email.
        cc (str): CC email addresses (comma-separated).
        bcc (str): BCC email addresses (comma-separated).
    """
    # Construct the mailto link
    base = "mailto:"
    params = []

    # Add recipient
    if to:
        params.append(f"{to}")

    # Add subject
    if subject:
        params.append(f"subject={quote(subject)}")

    # Add body
    if body:
        params.append(f"body={quote(body)}")

    # Add CC
    if cc:
        params.append(f"cc={quote(cc)}")

    # Add BCC
    if bcc:
        params.append(f"bcc={quote(bcc)}")

    # Join parameters with '&'
    mailto_link = "mailto:" + '&'.join(params)

    # Open the default email client
    return mailto_link

# Example usage
link = send_email(
    to='example@example.com',
    subject='Meeting Reminder',
    body='Just a quick reminder about our meeting tomorrow.',
    cc='manager@example.com',
    bcc='admin@example.com'
)

link

'mailto:example@example.com&subject=Meeting%20Reminder&body=Just%20a%20quick%20reminder%20about%20our%20meeting%20tomorrow.&cc=manager%40example.com&bcc=admin%40example.com'